In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing

In [ ]:
train=pd.read_csv('/kaggle/input/titanic/train.csv')
test= pd.read_csv("/kaggle/input/titanic/test.csv")


In [ ]:
train.head(10)

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train['Age'].fillna(train['Age'].mean(), inplace=True)
test['Age'].fillna(test['Age'].mean(), inplace=True)
train.info()

In [ ]:
train.drop(['Name','Ticket','Cabin'], axis=1, inplace=True)
test.drop(['Name','Ticket','Cabin'], axis=1, inplace=True)

In [ ]:
test['Fare'].fillna(test['Fare'].mean(), inplace=True)

In [ ]:
train.head()

In [ ]:
def bar_chart(feature):
    survived = train[train['Survived']==1][feature].value_counts()
    dead = train[train['Survived']==0][feature].value_counts()
    df = pd.DataFrame([survived,dead])
    df.index = ['Survived','Dead']
    df.plot(kind='bar',stacked=True, figsize=(10,5))

In [ ]:
bar_chart('Sex')

In [ ]:
bar_chart('Pclass')

In [ ]:
bar_chart('SibSp')

In [ ]:
bar_chart('Parch')

In [ ]:
def process_age(df,cut_points,label_names):
    df["Age_categories"] = pd.cut(df["Age"],cut_points,labels=label_names)
    return df

cut_points = [0,5,12,18,35,60,100]
label_names = ["Infant","Child","Teenager","Young Adult","Adult","Senior"]

train = process_age(train,cut_points,label_names)
test = process_age(test,cut_points,label_names)

pivot = train.pivot_table(index="Age_categories",values='Survived')
pivot.plot.bar()
plt.show()


In [ ]:
train.head()

In [ ]:
train['Sex'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
test['Sex'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
train.head()

In [ ]:
X = train[['SibSp','Parch','Age', 'Sex', 'Pclass','Fare']].values
X[0:5]

In [ ]:
X= preprocessing.StandardScaler().fit(X).transform(X)

In [ ]:
X_testdata= test[['SibSp','Parch','Age', 'Sex', 'Pclass','Fare']].values
X_testdata[0:5]

In [ ]:
X_testdata= preprocessing.StandardScaler().fit(X_testdata).transform(X_testdata)
X_testdata[0:5]

In [ ]:
y = train["Survived"]
y[0:5]

In [ ]:
from sklearn.model_selection import train_test_split
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.1, random_state=3)
print(X_trainset.shape,y_trainset.shape)

# ****Decision tree****

In [ ]:
from sklearn.tree import DecisionTreeClassifier
titanic_Tree = DecisionTreeClassifier(criterion="entropy", max_depth = 5)
titanic_Tree # it shows the default parameters

In [ ]:
titanic_Tree.fit(X_trainset,y_trainset)

In [ ]:
predTree = titanic_Tree.predict(X_testset)

In [ ]:
print (predTree [0:5])
print (y_testset [0:5])


In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_testset, predTree))

# ****Logistic Regression****

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.1, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='saga').fit(X_train,y_train)
LR

In [ ]:
yhat = LR.predict(X_test)

yhat_prob = LR.predict_proba(X_test)

In [ ]:
from sklearn import metrics
print("logistic regression Accuracy: ", metrics.accuracy_score(y_test,yhat))

# ****SVM Classifier****

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train) 

In [ ]:
yhat = clf.predict(X_test)
yhat [0:5]

In [ ]:
from sklearn import metrics
print("SVM's Accuracy: ", metrics.accuracy_score(y_test,yhat))

# ****Testing****

In [ ]:
clf = svm.SVC(kernel='rbf')
clf.fit(X_train,y_train)

prediction = clf.predict(X_testdata)

In [ ]:

submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": prediction
    })

submission.to_csv('submission.csv', index=False)